<a href="https://colab.research.google.com/github/davidolima/car-model-classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho final - Curso Deep Learning
### Prof. Luciano de Rebouças
### Aluno: David de Oliveira Lima

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [2]:
# !kaggle datasets download -d peterjun/car196 -p "/content/drive/MyDrive/Colab Notebooks"
# %cd "/content/drive/MyDrive/Colab Notebooks"
# !unzip car196.zip
# !rm car196.zip

In [3]:
# Core
import tensorflow as tf
import tensorflow.keras as keras

# Redes e optimizers
from keras.applications import VGG19, ResNet50V2, EfficientNetB0, InceptionV3
from keras.optimizers import Adam, SGD

# Keras
from keras import Sequential
from keras.utils import to_categorical, image_dataset_from_directory, split_dataset
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import sparse_categorical_crossentropy, categorical_crossentropy
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K


# Etc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow_datasets as tfds
import scipy.io
# import deeplake

# Parâmetros

In [4]:
n_classes = 196
lr_sgd = .001
lr_adam = .001
epochs = 50
batch_size = 32
n_splits = 5
img_size = (256,256)
es_patience = 8
root = "/content/drive/MyDrive/Colab Notebooks/car196/"
val_split = 0.2
channels = 3

# Inicialização

In [5]:
adam = Adam(lr_adam,
           beta_1=0.9,
           beta_2=0.999,
           )

sgd = SGD(
    lr_sgd,
    momentum=.6
)
criterion = categorical_crossentropy
early_stop = EarlyStopping(monitor='val_loss',
                           mode='min', verbose=1,
                           patience=es_patience,
                           restore_best_weights = True
                           )

In [6]:
def plothist(hist):
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

## Data

In [7]:
# Carregando dataset
ds_train, ds_val = image_dataset_from_directory(
    root + "train/",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    seed=2023,
    validation_split=val_split,
    subset='both',
    image_size=img_size,
    shuffle=True,
)

print("Test images")
ds_test = image_dataset_from_directory(
    root + "test/",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    seed=2023,
    image_size=img_size,
    shuffle=True,
)

# ds_train, ds_val = split_dataset(ds_train, left_size=1-val_split, right_size=val_split)
#print(ds_train.,"training images")
# print(len(ds_val),"validation images")
# print(len(ds_test),"testing images")

Found 8144 files belonging to 196 classes.
Using 6516 files for training.
Using 1628 files for validation.
Test images
Found 8041 files belonging to 196 classes.


In [8]:
# Augmentation
resize_and_rescale = tf.keras.Sequential([
  Resizing(img_size[0], img_size[1]),
  Rescaling(1./255)
])

augmentation = keras.Sequential([
    # Resizing(img_size[0],img_size[1]),
    # Rescaling(1./255),
    RandomRotation(0.45),
    RandomFlip("horizontal"),
    RandomTranslation(height_factor=0.1, width_factor=0.1),
    ])

# Treinamento das redes

## VGG19

In [9]:
base = VGG19(
    include_top=False,
    weights="imagenet", # transfer learning da imagenet
    input_tensor=None,
    input_shape=(img_size[0],img_size[1],3),
    pooling=None,
    classes=n_classes,
    classifier_activation='softmax',
)

model_1 = Sequential([
    resize_and_rescale,
    augmentation,
    base,
    Flatten(),
    # Dense(1024,activation='relu',input_dim=512),
    # Dropout(.3),
    Dense(512, activation='relu'),
    Dropout(.2),
    Dense(256, activation='relu'),
    Dropout(.1),
    Dense(n_classes, activation='softmax'),
])
# vgg.add(base)
# vgg.add(Flatten())

# vgg.add(Dense(1024, activation=hl_act_fn,input_dim=512))
# vgg.add(Dropout(.3))
# vgg.add(Dense(512, activation='relu'))
# vgg.add(Dropout(.2))
# vgg.add(Dense(256, activation='relu'))
# vgg.add(Dropout(.1))
# vgg.add(Dense(n_classes, activation='softmax'))
model_1.build((None, img_size[0], img_size[1], channels))
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 8, 8, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 512)               16777728  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)              

In [ ]:
# Training
checkpoint = ModelCheckpoint(f"/content/trab-final-best-vgg.hdf5",
                            monitor='loss', verbose=1,
                            save_best_only=True,
                            mode='auto')

model_1.compile(loss=criterion,
            optimizer=sgd,
            metrics=["accuracy"],
            )

history = model_1.fit(ds_train,
                  validation_data=ds_val,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[keras.callbacks.History(), early_stop, checkpoint])

Epoch 1/50
126/204 [=================>............] - ETA: 49s - loss: 5.3235 - accuracy: 0.0042

In [ ]:
plothist(history)

In [ ]:
model_test_acc=model_1.evaluate(ds_test,batch_size=4)

## EfficientNetB0

In [ ]:
base = EfficientNetB0(
    include_top=False,
    weights="imagenet", # transfer learning da imagenet
    input_tensor=None,
    input_shape=(img_size[0],img_size[1],3),
    pooling=None,
    classes=n_classes,
    classifier_activation='softmax',
)

base.trainable = False
model_2 = Sequential([
    resize_and_rescale,
    augmentation,
    base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.2),
    Dense(256, activation='relu'),
    Dropout(.1),
    Dense(n_classes, activation='softmax'),
])

model_2.build((None, img_size[0], img_size[1], channels))
model_2.summary()

In [ ]:
# Training
checkpoint = ModelCheckpoint(f"/content/drive/MyDrive/Colab Notebooks/trab-final-best-effnetb0.hdf5",
                            monitor='loss', verbose=1,
                            save_best_only=True,
                            mode='auto')

model_2.compile(loss=criterion,
            optimizer=sgd,
            metrics=["accuracy"],
            )

history = model_2.fit(ds_train,
                  validation_data=ds_val,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[keras.callbacks.History(), early_stop, checkpoint])

In [ ]:
plothist(history)

In [ ]:
model_test_acc=model_2.evaluate(ds_test,batch_size=4)

## ResNet50V2

In [ ]:
base = ResNet50V2(
    include_top=False,
    weights="imagenet", # transfer learning da imagenet
    input_tensor=None,
    input_shape=(img_size[0],img_size[1],3),
    pooling=None,
    classes=n_classes,
    classifier_activation='softmax',
)

base.trainable = False
model_3 = Sequential([
    resize_and_rescale,
    augmentation,
    base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.2),
    Dense(256, activation='relu'),
    Dropout(.1),
    Dense(n_classes, activation='softmax'),
])

model_3.build((None, img_size[0], img_size[1], channels))
model_3.summary()

In [ ]:
# Training
checkpoint = ModelCheckpoint(f"/content/drive/MyDrive/Colab Notebooks/trab-final-best-resnet50v2.hdf5",
                            monitor='loss', verbose=1,
                            save_best_only=True,
                            mode='auto')

model_3.compile(loss=criterion,
            optimizer=sgd,
            metrics=["accuracy"],
            )

history = model_3.fit(ds_train,
                  validation_data=ds_val,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[keras.callbacks.History(), early_stop, checkpoint])

In [ ]:
plothist(history)

In [ ]:
model_test_acc=model_3.evaluate(ds_test,batch_size=4)

## InceptionV3

In [ ]:
base = InceptionV3(
    include_top=False,
    weights="imagenet", # transfer learning da imagenet
    input_tensor=None,
    input_shape=(img_size[0],img_size[1],3),
    pooling=None,
    classes=n_classes,
    classifier_activation='softmax',
)

base.trainable = False
model_4 = Sequential([
    resize_and_rescale,
    augmentation,
    base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.2),
    Dense(256, activation='relu'),
    Dropout(.1),
    Dense(n_classes, activation='softmax'),
])

model_4.build((None, img_size[0], img_size[1], channels))
model_4.summary()

In [ ]:
# Training
checkpoint = ModelCheckpoint(f"/content/drive/MyDrive/Colab Notebooks/trab-final-best-inceptionv3.hdf5",
                            monitor='loss', verbose=1,
                            save_best_only=True,
                            mode='auto')

model_4.compile(loss=criterion,
            optimizer=sgd,
            metrics=["accuracy"],
            )

history = model_2.fit(ds_train,
                  validation_data=ds_val,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[keras.callbacks.History(), early_stop, checkpoint])

In [ ]:
plothist(history)

In [ ]:
model_test_acc=model_4.evaluate(ds_test,batch_size=4)

# K-fold Cross-validation

In [ ]:
k = 5

# Referências

[1] - [Choose an activation function for deep learning - Machine Learning Mastery](https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/)

[2] - [Data augmentation - Tensorflow](https://www.tensorflow.org/tutorials/images/data_augmentation)
